<a href="https://colab.research.google.com/github/CamilleGreen5/Deep_Learning_with_Python/blob/master/Classification_dog_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LOAD DATA**

In [1]:
# !pip install -q kaggle

In [7]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"camillegreen5","key":"15f4c19bac781a32fae16245218a1f39"}'}

In [8]:
# !rm -r sample_data
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!rm kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets list

In [ ]:
!kaggle competitions download -c 'dogs-vs-cats'

In [ ]:
!unzip train.zip
!unzip test1.zip
!rm train.zip
!rm test1.zip

In [11]:
!mkdir data
!mkdir data/train_data
!mkdir data/train_data/dog
!mkdir data/train_data/cat
!mkdir data/val_data
!mkdir data/val_data/dog
!mkdir data/val_data/cat
!mkdir data/test_data
!mkdir data/test_data/dog
!mkdir data/test_data/cat

train_dir = "data/train_data"
val_dir = "data/val_data"
test_dir = "data/test_data"

train_dog_dir = "data/train_data/dog"
train_cat_dir = "data/train_data/cat"
val_dog_dir = "data/val_data/dog"
val_cat_dir = "data/val_data/cat"
test_dog_dir = "data/test_data/dog"
test_cat_dir = "data/test_data/cat"

In [12]:
list_dog = glob.glob("./train/dog*")
list_cat = glob.glob("./train/cat*")
list_train_dog = list_dog[:2000]
list_train_cat = list_cat[:2000]
list_val_dog = list_dog[2000:3000]
list_val_cat = list_cat[2000:3000]
# print(len(list_train_dog), len(list_val_dog))
# print(len(list_train_cat), len(list_val_cat))
for path in list_train_dog:
    shutil.copy(path, train_dog_dir)
    os.remove(path)
for path in list_train_cat:
    shutil.copy(path, train_cat_dir)
    os.remove(path)
for path in list_val_cat:
    shutil.copy(path, val_cat_dir)
    os.remove(path)
for path in list_val_dog:
    shutil.copy(path, val_dog_dir)
    os.remove(path)

In [17]:
!ls data/val_data/dog | wc -l

1000


## **MODEL**

In [47]:
import os
import glob
import shutil
import tensorflow as tf
import datetime
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16

In [48]:
def show_example(nb_ex=4):
    fnames = [os.path.join(train_cat_dir, fname) for fname in os.listdir(train_cat_dir)]
    img_path = fnames[10]
    img =image.load_img(img_path, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.reshape(x, (1,) + np.shape(x))
    print(np.shape(x))
    i = 0
    for batch in train_datagen.flow(x, batch_size=1):
        plt.figure(i)
        imgplot = plt.imshow(image.array_to_img(batch[0]))
        i += 1
        if i % nb_ex == 0:
            break

In [49]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [56]:
def build_model():

    conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    conv_base.trainable = False
    
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])
    return model

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, \
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, \
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=20, class_mode='binary')


In [ ]:
def unfreeze(model):
    set_trainable = False
    for layer in model.layers:
        if layer.name == 'block5_conv1':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])
    return model

In [ ]:
new_model = False

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

if new_model:
    model = build_model()
else:
    model = tf.keras.models.load_model("cat_and_dog_classifier.h5")
    model = unfreeze(model)

history = model.fit(train_generator, steps_per_epoch=100, epochs=100, validation_data=validation_generator, \
                    validation_steps=50, callbacks=[tensorboard_callback])
model.save("cat_and_dog_classifier.h5")

Epoch 1/30
100/100 [==============================] - 24s 238ms/step - loss: 0.6008 - acc: 0.6815 - val_loss: 0.5273 - val_acc: 0.7700
Epoch 2/30
100/100 [==============================] - 23s 235ms/step - loss: 0.4849 - acc: 0.7860 - val_loss: 0.4423 - val_acc: 0.8090
Epoch 3/30
100/100 [==============================] - 24s 238ms/step - loss: 0.4310 - acc: 0.8030 - val_loss: 0.4230 - val_acc: 0.8160
Epoch 4/30
100/100 [==============================] - 24s 236ms/step - loss: 0.4146 - acc: 0.8190 - val_loss: 0.3886 - val_acc: 0.8230
Epoch 5/30
100/100 [==============================] - 24s 235ms/step - loss: 0.3940 - acc: 0.8275 - val_loss: 0.4055 - val_acc: 0.8170
Epoch 6/30
100/100 [==============================] - 24s 237ms/step - loss: 0.3888 - acc: 0.8315 - val_loss: 0.3583 - val_acc: 0.8480
Epoch 7/30
100/100 [==============================] - 23s 235ms/step - loss: 0.3704 - acc: 0.8415 - val_loss: 0.3704 - val_acc: 0.8360
Epoch 8/30
100/100 [==============================] - 2

In [ ]:
%tensorboard --logdir logs/fit --host localhost